In [3]:
import os
import numpy as np

In [4]:
import os
import torch
import glob
import numpy as np 
import random
import math
from os import listdir
from os.path import isfile, join
from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n
from torch_geometric.data import Data

def bump(g):
    return g
    # return Data.from_dict(g.__dict__)

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
      prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
      prot_1 = bump(prot_1)
      prot_2 = bump(prot_2)
      return prot_1, prot_2, torch.tensor(self.label[index])

In [5]:
npy_file = "../human_features/npy_file_new(human_dataset).npy"
npy_file_test = "../human_features/npy_file_new(human_dataset)_test.npy"
processed_dir="../human_features/esm/"
# train = LabelledDataset(npy_file=npy_file, processed_dir=processed_dir)
# test = LabelledDataset(npy_file=npy_file_test, processed_dir=processed_dir)

In [6]:
from tqdm.auto import tqdm

X_train = []
y_train = []
mapping_train = np.load(npy_file)
bad_rows = []
for row in tqdm(mapping_train):
    try:
        _, _, prot1, _, _, prot2, label = row
        prot_1 = os.path.join(processed_dir, prot1+".pt")
        prot_2 = os.path.join(processed_dir, prot2+".pt")
        prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
        prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
        p = torch.cat([prot_1, prot_2])
        label = float(label)
        X_train.append(p)
        y_train.append(label)
    except:
        bad_rows.append(row)

  0%|          | 0/98396 [00:00<?, ?it/s]

In [30]:
X_test = []
y_test = []
bad_test_rows = []
mapping_test = np.load(npy_file_test)
for row in tqdm(mapping_test):
    try:
        _, _, prot1, _, _, prot2, label = row
        prot_1 = os.path.join(processed_dir, prot1+".pt")
        prot_2 = os.path.join(processed_dir, prot2+".pt")
        prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
        prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
        p = torch.cat([prot_1, prot_2])
        label = float(label)
        X_test.append(p)
        y_test.append(label)
    except:
        bad_test_rows.append(row)

  0%|          | 0/5677 [00:00<?, ?it/s]

In [8]:
for i in range(len(X_train)):
    X_train[i] = X_train[i][None, :]

In [9]:
X_train = torch.cat(X_train)

In [11]:
!pip3 install -qq scikit-learn 

In [12]:
from sklearn.model_selection import train_test_split

train, val, ytrain, yval = train_test_split(X_train, y_train, test_size=0.2)

In [13]:
ytrain = [x for x in ytrain]
yval = [x for x in yval]

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)

model = nn.Sequential(
    nn.Linear(in_features=X_train.shape[1], out_features=1)
)

criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

dataset = TensorDataset(X_train_torch, y_train_torch)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

num_epochs = 10

for epoch in tqdm(range(num_epochs)):
    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        logits = model(X_batch).squeeze()
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    logits = model(X_train_torch).squeeze(dim=1)
    probs = torch.sigmoid(logits)
    print("Example predictions:", probs[:10])

/tmp/ipykernel_524/3616782837.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_torch = torch.tensor(X_train, dtype=torch.float32)


  0%|          | 0/10 [00:00<?, ?it/s]

Example predictions: tensor([0.4317, 0.5838, 0.2083, 0.3097, 0.2952, 0.3907, 0.2535, 0.3899, 0.3198,
        0.4608])


In [17]:
import torch
import torch.nn as nn

def validate_model_mse(val_features, val_probs, model):
    X_val_torch = torch.tensor(val_features, dtype=torch.float32)
    y_val_torch = torch.tensor(val_probs, dtype=torch.float32)
    with torch.no_grad():
        logits = model(X_val_torch).squeeze(dim=1)
        preds = torch.sigmoid(logits)
        mse_value = ((preds - y_val_torch) ** 2).mean()
    print(f"Validation MSE: {mse_value.item():.6f}")
    return preds

In [28]:
validate_model_mse(val, yval, model);

/tmp/ipykernel_524/3830430384.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_torch = torch.tensor(val_features, dtype=torch.float32)


Validation MSE: 0.021742


In [31]:
for i in range(len(X_test)):
    X_test[i] = X_test[i][None, :]
X_test = torch.cat(X_test)
y_test = [x for x in y_test]

X_test = X_test.squeeze(1)

In [32]:
validate_model_mse(X_test, y_test, model);

Validation MSE: 0.027581


/tmp/ipykernel_524/3830430384.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_torch = torch.tensor(val_features, dtype=torch.float32)


In [63]:
from sklearn.linear_model import LogisticRegression

In [64]:
logreg = LogisticRegression().fit(train, ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(yval, logreg.predict(val))

0.9602628763564114

In [79]:
accuracy_score(y_test, logreg.predict(X_test))

0.9393564356435643

In [80]:
from collections import Counter
Counter(y_test)

Counter({True: 5203, False: 453})

In [82]:
5203 / (453 + 5203) 

0.9199080622347949